<a href="https://colab.research.google.com/github/ObaidAshraf/GenAI_AgenticAI/blob/master/langchain_rag_with_docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain langchain-pinecone langchain-google-genai pinecone-client langchain-community pypdf

In [11]:
import os

from google.colab import userdata
pinecone_api_key = userdata.get('PINECONE_API_KEY')
google_api_key = userdata.get('GOOGLE_API_KEY')

if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = google_api_key

if not os.getenv("PINECONE_API_KEY"):
  os.environ['PINECONE_API_KEY'] = pinecone_api_key

In [3]:
# Import the Pinecone library
from pinecone import Pinecone
from pinecone import ServerlessSpec

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key=pinecone_api_key)

In [4]:
# Create a dense index with integrated embedding
index_name = "online-web-project"
existing_indexes = [index.name for index in pc.list_indexes()]
if index_name not in existing_indexes:
  pc.create_index(
    name=index_name,
    dimension=3072,
    metric="cosine",
    spec=ServerlessSpec(
      cloud="aws",
      region="us-east-1"
    )
  )

index = pc.Index(index_name)

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [ ]:
# from langchain_pinecone import Pinecone as PineconeVectorStore

# vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
client = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash-lite",
)

In [7]:
response = client.invoke("What is the work experience of Obaid Ashraf")
response.content

'Unfortunately, without more context, it\'s impossible for me to provide specific details about the work experience of "Obaid Ashraf."\n\nHere\'s why and what you might need to do to find this information:\n\n**Why I Can\'t Tell You Directly:**\n\n*   **Privacy:** Personal work experience is generally not publicly available unless the individual chooses to share it.\n*   **Ambiguity:** "Obaid Ashraf" is a name, and there could be many individuals with that name. I have no way to differentiate between them.\n*   **Lack of Access:** I don\'t have access to private databases of employment history or personal résumés.\n\n**How You Might Find This Information (Depending on Your Goal):**\n\n1.  **If you know the specific Obaid Ashraf you\'re looking for:**\n    *   **LinkedIn:** This is the most common platform for professionals to list their work experience. Search for "Obaid Ashraf" on LinkedIn. You may need to add more details like their location, company, or industry to narrow down the s

In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("Obaid_Ashraf.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks = text_splitter.split_documents(docs)

Document(metadata={'producer': 'Skia/PDF m142 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Obaid_Ashraf.docx', 'source': 'Obaid_Ashraf.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='MUHAMMAD  OBAID  ASHRAF  Phone :  +923142142360  Email:  ubaidashraf91@gmail.com GitHub :https://github.com/obaidAshraf/ LinkedIn :  https://www.linkedin.com/in/obaidashraf91/  Profile:  Passionate,  skilled  and  experienced  Software  Engineer,  currently  working  in  the  field  of  \nFinTech\n \nSoftware\n \ndevelopment,\n \nwith\n \na\n \npassion\n \nto\n \ndevelop\n \nefficient\n \nand\n \nscalable\n \nsolutions.\n \nSkilled\n \nin\n \nEmbedded\n \nsoftware\n \ndevelopment\n \nincluding\n \nLinux-based\n \ndevice\n \ndrivers\n \nand\n \nuser-space\n \ntools,\n \nFirmware\n \ndevelopment\n \nusing\n \nOpenWRT\n \nfor\n \nRouters\n \nand\n \nAccess-Points\n \nand\n \nalso\n \nproficient\n \nin\n \nFull-stack\n \nWeb\n \nDevelopment\n \nand\n \ncross-pla

In [12]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=embeddings,
    index_name=index_name
)


In [22]:
query = "C++ experience?"
results = vector_store.similarity_search_with_score(query, k=1)

for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")


* [SIM=0.678978] TECHNICAL  SKILLS:  ●  Programming  languages   C,  C++,  Bash,  Python,  Lua/Luci,  HTML5,  Javascript,  PHP,   
 
 
 
 
MySQL,Bootstrap,
 
CSS3,
 
NodeJS,
 
ReactJS,
 
React
 
Native,
 
 
 
 
 
MongoDB,
 
Flask/Django,
 
GraphQL,
 
Docker
 
 EDUCATION:  NED  University  of  Engineering  &  Technology,  Karachi  -  MS  (Computer  Science)  01/2018  -  06/2020         CGPA:  3.85/4.0  ●  Achieved  Second  Position  in  the  batch   Sir  Syed  University  of  Engineering  &  Technology,  Karach i  -  BS  (Electronic  Engineering)  01/2011  -  12/2014         CGPA:  3.78/4.0  ●  Gold  Medalist  (Best  graduate  of  the  batch)  ●  Automatic  Health  &  Care  Unit  (Final  Year  Project)  ●  Awarded  Merit  Scholarships  in  every  semester   PUBLICATIONS:  ●  Peer-reviewed  Journal  Article:  o  Obaid  Ashraf,  Automatic  Health  Care  Unit,  International  Journal  of  
Endorsing
 
Health
 
Science
 
Research(IJEHSR)-AEIRC
,
 
Volume
 
3,
 
(
22-26) [{'creationdate': ''

In [29]:
def answer_to_user(query: str) -> str:
    vector_results = vector_store.similarity_search_with_score(query)

    answers = client.invoke(f"Answer to the query: {query} using references {vector_results}")
    print(f"Answer: {answers.content}")


answer_to_user("What is the github repo url of Obaid Ashraf?")

Answer: The GitHub repo URL of Obaid Ashraf is https://github.com/obaidAshraf/.
